<h1 style="color:green; font-size:30px; font-weight:700; text-align:center">
    <u><b>Part 2: TMDB Api</b></u>  
</h1> 

<p></p> 
<strong> Author:</strong> Gurtej Bains <br> 
<strong> Data Source:</strong> https://www.imdb.com/interfaces/  <br> 
<strong> Last Updated: </strong>  <br>  
<strong> Problem Statement:</strong> <br>

<h1 style="color:green"><b>Importing Modules</b> </h1>

## Import Libraries

In [1]:
# import libraries  

# Numpy
import numpy as np

# Pandas
import pandas as pd

# MatPlotLib
import matplotlib.pyplot as plt

# Seaborn
import seaborn as sns

# MissingNo
import missingno as msno

# SQL related 
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# for directory 
import os

# TQDM
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

## API Modules

In [2]:
import json
with open('/Users/gurte/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

## Color class

In [4]:
# Color class to make print look pretty  
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello, World!' + color.END)

Hello, World!


<h1 style="color:green"><b>API Calls</b> </h1>

## Define Function: Certification data 

In [5]:
# Function to obtain movie rating
def get_movie_with_rating(movie_id):
    # Get movie and release dates
    movie = tmdb.Movies(movie_id)
    # Construct output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

## Test Function: Certification data 

In [6]:
## testing our function by looping through a list of ids
import pandas as pd
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
results = pd.DataFrame(results)
results.head(2)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28264,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28264,PG-13


In [7]:
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 1


[['tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=03a3ec891d024b34bc833be7e1c4a979')]]

## Define Function: Append data

In [8]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

<h1 style="color:green"><b>Data and folders</b> </h1>

## Folder

In [9]:
FOLDER = "clean data/"

os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

## Get 'basics' data

In [10]:
# Load in the IMDB title_basics file
basics = pd.read_csv("clean data/title_basics.csv.gz")
basics.head(2)

,tconst,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama


## Build loop

In [18]:
# year list 
YEARS_TO_GET = ['2000', '2001'] 

# Error
errors = []

In [12]:
####### OUTER loop ########
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Define the JSON file to store results for current Year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    
    # Save an empty dict with just "imdb_id" to the new JSON file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
               
    # Filter the IMDB title_basics file on current Year
    df = basics.loc[ basics['startyear']==YEAR].copy()
    
    # Save IMDB title_basics file movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Create a dataframe from the JSON file
    previous_df = pd.read_json(JSON_FILE)
    
    # Filter out movie ids that are already exist in the JSON file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

###### INNER Loop #######
    # Get index and current movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve the data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to JSON file function
            write_json(temp,JSON_FILE)
            
            # Pause 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails, make a dict with just the id and None for certification.
        except Exception as e:
            continue
            
    # Save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                         compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000: 0it [00:00, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]

<h1 style="color:green"><b>EDA Data</b> </h1>

## Get data

In [13]:
# Load in the csv.gz's of results for years 2000 and 2001
year_2000 = pd.read_csv("clean data/final_tmdb_data_2000.csv.gz")
year_2001 = pd.read_csv("clean data/final_tmdb_data_2001.csv.gz")

In [14]:
# Combine years 2000 and 2001 into a single dataframe
tmdb_results_combined_df = year_2000.append(year_2001)

In [15]:
# Display the tail and head of the combines dataframe
tmdb_results_combined_df.head(2)

,imdb_id
0,0
0,0


In [16]:
# Save a final merged .csv.gz of all of the tmdb api data 
tmdb_results_combined_df.to_csv("clean data/tmdb_results_combined.csv.gz", 
                                compression="gzip", 
                                index=False)

## Movies with some valid financial info

In [17]:
# filter data 
filter_df = tmdb_results_combined_df[(tmdb_results_combined_df['budget'] > 0) | (tmdb_results_combined_df['revenue'] > 0)]

print (f'\nThere are a total of {color.BOLD}{color.RED}{len(filter_df)}{color.END}{color.END} movies with some valid financial info\n') 

KeyError: 'budget'

## Certification counts

In [ ]:
filter_df.certification.value_counts()

## Average revenue per certification 

In [ ]:
filter_df.groupby(['certification']).aggregate({'revenue':'mean'}).sort_values(by='revenue', ascending=False)\
.head(10).style.format('${:,.0f}')

## Average budget by certification

In [ ]:
filter_df.groupby(['certification']).aggregate({'budget':'mean'}).sort_values(by='budget', ascending=False)\
.head(10).style.format('${:,.0f}')